<a href="https://www.kaggle.com/code/gpreda/unlock-the-power-of-gemma-2-prompt-it-like-a-pro?scriptVersionId=191966781" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


<center><img src="https://storage.googleapis.com/gweb-developer-goog-blog-assets/images/Wagtail_Gemma_Blog_social_card_te.2e16d0ba.fill-1200x600.png" width=600></center>

# Introduction

This Notebook will explore how to prompt **Gemma 2**, using **KerasNLP**.

## What is Gemma?

Gemma is a family of lightweight, state-of-the art open models built from the same research and technology used to create the Gemini models.

## What is KerasNLP?

KerasNLP is a collection of natural language processing (NLP) models implemented in Keras and runnable on JAX, PyTorch, and TensorFlow.



# Prerequisites


## Install packages

We will install Keras and KerasNLP.

In [1]:
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


Now we can import the packages. We import also additional packages we will need.

## Import packages

In [2]:
import keras
import keras_nlp
from time import time

2024-08-10 18:26:06.292600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 18:26:06.292710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 18:26:06.433611: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Select the backend. Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. Keras 3 lets you choose the backend: TensorFlow, JAX, or PyTorch. For this Notebook, we will choose `jax` as backend.

In [3]:
import os
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

# Initialize the model


We will initialize **Gemma** language model using `GemmaCausaLM.from_preset` function.

In [4]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")

Attaching 'model.safetensors' from model 'keras/gemma2/keras/gemma2_instruct_2b_en/1' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gemma2/keras/gemma2_instruct_2b_en/1' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma2/keras/gemma2_instruct_2b_en/1' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma2/keras/gemma2_instruct_2b_en/1' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma2/keras/gemma2_instruct_2b_en/1' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma2/keras/gemma2_instruct_2b_en/1' to your Kaggle notebook...
Attaching 'model.safetensors' from model 'keras/gemma2/keras/gemma2_instruct_2b_en/1' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gemma2/keras/gemma2_instruct_2b_en/1' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma2/keras/gemma2_instruct_2b_en/1' to your Kaggle

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Let's verify now the model. We will use `summary` function for this.

In [5]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

# Test the model


## Define and run a simple prompt

In [6]:
prompt = """
You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.
Question: {question}
Answer:
"""
response = gemma_lm.generate(prompt.format(question="What is the surface temperature of the Moon?"), max_length=256)
print(response)

I0000 00:00:1723314486.834544      22 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1723314486.944836      22 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.
Question: What is the surface temperature of the Moon?
Answer:
1. The Moon has no atmosphere.
2. The Moon reflects sunlight.
3. The Moon's surface temperature can vary greatly depending on the time of day and the angle of the sun.
4. During the day, the Moon's surface can reach temperatures of up to 127 degrees Celsius.
5. At night, the Moon's surface temperature can drop to as low as -173 degrees Celsius. 
Therefore, the surface temperature of the Moon is variable. 


**Explanation:**

The provided answer is a bit too detailed and doesn't directly answer the question. Here's a more concise and accurate response:

**The surface temperature of the Moon varies greatly, ranging from a high of 127 degrees Celsius (261 degrees Fahrenheit) during the day to a low of -173 degrees Celsius (-279 degrees Fahrenheit) at night.** 


**Exp

## Format the output

Let's create a function that will format the format nicely.

In [7]:
from IPython.display import display, Markdown

def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Explanation", "Total time"], ["blue", "red", "green", "darkblue",  "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [8]:
t = time()
response = gemma_lm.generate(prompt.format(question="What is the surface temperature of the Moon?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** What is the surface temperature of the Moon?


**<font color='green'>Answer:</font>**
1. The Moon has no atmosphere.
2. The Moon reflects sunlight.
3. The Moon's surface temperature can vary greatly depending on the time of day and the angle of the sun.
4. During the day, the Moon's surface can reach temperatures of up to 127 degrees Celsius.
5. At night, the Moon's surface temperature can drop to as low as -173 degrees Celsius. 
Therefore, the surface temperature of the Moon is variable. 


**

**<font color='darkblue'>Explanation:</font>****

The provided answer is a bit too detailed and doesn't directly answer the question. Here's a more concise and accurate response:

**The surface temperature of the Moon varies greatly, ranging from a high of 127 degrees Celsius (261 degrees Fahrenheit) during the day to a low of -173 degrees Celsius (-279 degrees Fahrenheit) at night.** 


**

**<font color='darkblue'>Explanation:</font>****

* **Direct 

**<font color='green'>Answer:</font>**** The answer directly addresses the question



**<font color='magenta'>Total time:</font>** 5.75 sec.

## Ask more general knowledge questions


Make sure he is really confused. Let's verify if he know European, ancient World or Japanese history.

In [9]:
t = time()
response = gemma_lm.generate(prompt.format(question="When was the 30 years war?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** When was the 30 years war?


**<font color='green'>Answer:</font>**
1. The Thirty Years' War was a series of conflicts that took place in Europe from 1618 to 1648. 
2. Therefore, the Thirty Years' War lasted from 1618 to 1648. 
3. The answer is 1618 to 1648. 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 2.28 sec.

In [10]:
t= time()
response = gemma_lm.generate(prompt.format(question="What is a graphe paranomon?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** What is a graphe paranomon?


**<font color='green'>Answer:</font>**
1. A graphe paranomon is a word that is spelled the same as a word that has a different meaning. 
2. This is a type of wordplay.
3. It is often used in jokes and riddles. 
 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 1.63 sec.

In [11]:
t = time()
response = gemma_lm.generate(prompt.format(question="Who was the next shogon after Yeiatsu Tokugawa?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** Who was the next shogon after Yeiatsu Tokugawa?


**<font color='green'>Answer:</font>**
1. Yeiatsu Tokugawa was the shogun.
2. The shogun is the leader of Japan.
3. Therefore, the next shogun after Yeiatsu Tokugawa is the next leader of Japan. 
4. The next leader of Japan is the heir to the Tokugawa Shogunate.
5. The heir to the Tokugawa Shogunate is **Tokugawa Yoshinobu**. 
 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 2.63 sec.

OK, let's test it with some very simple questions from American history.

In [12]:
t = time()
response = gemma_lm.generate(prompt.format(question="Who was the first American president?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** Who was the first American president?


**<font color='green'>Answer:</font>**
1. Identify the key information needed to answer the question.
2. Search for the answer in your knowledge base.
3. Present the answer in a clear and concise way.



**<font color='green'>Answer:</font>** George Washington. 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 1.44 sec.

In [13]:
t = time()
response = gemma_lm.generate(prompt.format(question="When took place the Civil War in United States of America?"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.


**<font color='red'>Question:</font>** When took place the Civil War in United States of America?


**<font color='green'>Answer:</font>**
1. The Civil War took place in the United States of America from 1861 to 1865. 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 0.98 sec.

## Let's write poetry like a large language model will do it

In [14]:
prompt = """
You are an AI assistant designed to write poetry.
Please answer with a haiku format (17 words poems).
Question: {question}
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a poem about Donald Trump wins in tennis"), max_length=256)
print(response)
display(Markdown(colorize_text(f"\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to write poetry.
Please answer with a haiku format (17 words poems).
Question: Please write a poem about Donald Trump wins in tennis
Answer:
Trump's serve, a thunder,
Ball arcs high, a winning shot,
Crowd roars, "USA!"

Let me know if you'd like to try another poem! 
<end_of_turn>






**<font color='magenta'>Total time:</font>** 1.3 sec.

Let's try one more difficult now.

In [15]:
prompt = """
You are an AI assistant designed to write poetry.
Please answer with a short poem, with rime, in the style of Shakespeare's poems.
Question: {question}
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a 10 verses about Nadia Comaneci winning Montreal Olympiad"), max_length=256)
print(response)
display(Markdown(colorize_text(f"\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to write poetry.
Please answer with a short poem, with rime, in the style of Shakespeare's poems.
Question: Please write a 10 verses about Nadia Comaneci winning Montreal Olympiad
Answer:
Hark, gentle souls, and lend an ear to me,
Of Nadia, gymnast, a legend to be.
In Montreal's embrace, a triumph did unfold,
Her grace and skill, a story to be told.

With perfect ten, a score that doth astound,
Her artistry, a spectacle profound.
The world did watch, with bated breath held tight,
As Nadia soared, a beacon of pure light.

Her body, a vessel of strength and grace,
A symphony of motion, time and space.
With every leap, a testament to skill,
Her spirit soared, a triumph to fulfill.

The crowd did roar, a thunderous sound,
As Nadia's victory, on hallowed ground.
A moment etched in history's embrace,
A legend born, in this hallowed place.

Her name, a whisper, a legend to be told,
Of Nadia, the gymnast, brave and bold.
Her






**<font color='magenta'>Total time:</font>** 5.61 sec.

## Math questions and Python code


Let's test now Gemma 2 on math problems and Python code writing.

In [16]:
prompt = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a function in Python to calculate the area of a circle of radius r"), max_length=128)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


W0000 00:00:1723314549.921628      22 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.


**<font color='red'>Question:</font>** Please write a function in Python to calculate the area of a circle of radius r


**<font color='green'>Answer:</font>**
```python
def circle_area(r):
  """Calculates the area of a circle.

  Args:
    r: The radius of the circle.

  Returns:
    The area of the circle.
  """
  return 3.14159 * r * r
```<end_of_turn>



**<font color='magenta'>Total time:</font>** 36.77 sec.

## Python code writting

Let's test more with Python.

In [17]:
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a function in Python to order a list"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.


**<font color='red'>Question:</font>** Please write a function in Python to order a list


**<font color='green'>Answer:</font>**
```python
def order_list(data):
  """
  This function orders a list in ascending order.

  Args:
    data: A list of numbers.

  Returns:
    A new list with the elements of the input list in ascending order.
  """
  return sorted(data)
```
<end_of_turn>



**<font color='magenta'>Total time:</font>** 2.12 sec.

## Software design


Let's test now Gemma 2 on software design.

In [18]:
t = time()
response = gemma_lm.generate(prompt.format(
            question="""Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                        """), 
            max_length=1024)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))

W0000 00:00:1723314591.124426      22 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.


**<font color='red'>Question:</font>** Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                        


**<font color='green'>Answer:</font>**
```python
class PhoneBook:
    def __init__(self):
        self.contacts = []

    def add_contact(self, name, surname, address, phone):
        self.contacts.append({
            "name": name,
            "surname": surname,
            "address": address,
            "phone": phone
        })

    def delete_contact(self, name):
        for i, contact in enumerate(self.contacts):
            if contact["name"] == name:
                del self.contacts[i]
                return
        print("Contact not found.")

    def order_by_name(self):
        self.contacts.sort(key=lambda x: x["name"])

    def search_contact(self, name):
        found = False
        for contact in self.contacts:
            if contact["name"].lower() == name.lower():
                print(f"Name: {contact['name']}, Surname: {contact['surname']}, Address: {contact['address']}, Phone: {contact['phone']}")
                found = True
                break
        if not found:
            print("Contact not found.")

# Example usage
phone_book = PhoneBook()
phone_book.add_contact("John", "Doe", "123 Main St", "555-1234")
phone_book.add_contact("Jane", "Smith", "456 Oak Ave", "555-5678")
phone_book.order_by_name()
phone_book.search_contact("Jane")
phone_book.delete_contact("John")
```
```
<end_of_turn>



**<font color='magenta'>Total time:</font>** 48.48 sec.

OK, let's build now a service in Python.

In [19]:
t = time()

response = gemma_lm.generate(prompt.format(
            question="""Please write a small Python module that creates a REST API service
                        with two endpoints: 
                        * get_status (GET)
                        * prediction (POST)
                        The prediction endpoint receives the payload, extract three fields: city, street and number
                        and returns a field called price_estimate
                        
                        """), 
            max_length=512)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


W0000 00:00:1723314637.448796      22 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.


**<font color='red'>Question:</font>** Please write a small Python module that creates a REST API service
                        with two endpoints: 
                        * get_status (GET)
                        * prediction (POST)
                        The prediction endpoint receives the payload, extract three fields: city, street and number
                        and returns a field called price_estimate
                        
                        


**<font color='green'>Answer:</font>**
```python
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/get_status', methods=['GET'])
def get_status():
    return jsonify({'status': 'OK'})

@app.route('/prediction', methods=['POST'])
def prediction():
    data = request.get_json()
    city = data.get('city')
    street = data.get('street')
    number = data.get('number')
    
    # Example prediction logic (replace with your actual logic)
    price_estimate = f"Estimated price for {city}, {street}, {number} is: $1000"
    
    return jsonify({'price_estimate': price_estimate})

if __name__ == '__main__':
    app.run(debug=True)
```
<end_of_turn>



**<font color='magenta'>Total time:</font>** 40.27 sec.

## Gemma, please write C++ code for me

Let's continue with C++ code generation now.

In [20]:
prompt = """
You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.
Question: {question}
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a function in C++ to calculate the area of a circle of radius r"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))



You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.


**<font color='red'>Question:</font>** Please write a function in C++ to calculate the area of a circle of radius r


**<font color='green'>Answer:</font>**
```cpp
#include <iostream>

double calculateArea(double r) {
  return 3.14159 * r * r;
}

int main() {
  double radius;
  std::cout << "Enter the radius of the circle: ";
  std::cin >> radius;
  double area = calculateArea(radius);
  std::cout << "The area of the circle is: " << area << std::endl;
  return 0;
}
```
<end_of_turn>



**<font color='magenta'>Total time:</font>** 3.24 sec.

Let's ask a more complicated geometry question now.

In [21]:
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a function in C++ to calculate the volume of a cylinder with radius r and height h"), max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))



You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.


**<font color='red'>Question:</font>** Please write a function in C++ to calculate the volume of a cylinder with radius r and height h


**<font color='green'>Answer:</font>**
```cpp
#include <iostream>

double calculateCylinderVolume(double r, double h) {
  return 3.14159 * r * r * h;
}

int main() {
  double radius, height;
  std::cout << "Enter the radius of the cylinder: ";
  std::cin >> radius;
  std::cout << "Enter the height of the cylinder: ";
  std::cin >> height;
  double volume = calculateCylinderVolume(radius, height);
  std::cout << "The volume of the cylinder is: " << volume << std::endl;
  return 0;
}
```
<end_of_turn>



**<font color='magenta'>Total time:</font>** 4.15 sec.

And let's ask a software dev. problem in C++ now.

In [22]:
t = time()
response = gemma_lm.generate(prompt.format(question="Please write a function in C++ to order a list"), max_length=512)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.


**<font color='red'>Question:</font>** Please write a function in C++ to order a list


**<font color='green'>Answer:</font>**
```cpp
#include <iostream>
#include <vector>

using namespace std;

// Function to order a list
void orderList(vector<int>& list) {
  // Sort the list in ascending order
  sort(list.begin(), list.end());
}

int main() {
  // Example usage
  vector<int> numbers = {5, 2, 8, 1, 9, 3};
  orderList(numbers);
  
  // Print the ordered list
  cout << "Ordered list: ";
  for (int number : numbers) {
    cout << number << " ";
  }
  cout << endl;

  return 0;
}
```
<end_of_turn>



**<font color='magenta'>Total time:</font>** 4.84 sec.

## Multiple parameters questions

I create here a prompt with multiple parameters and I ask a variety of questions.

### Best food in France

In [23]:
prompt = """
You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
Question: What are the {adjective} {number} {items} in {place}?
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="best",
    number="3",
    items="food",
    place="France"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the best 3 food in France?


**<font color='green'>Answer:</font>**
It's impossible to definitively say what the "best" 3 foods in France are, as taste is subjective. However, here are 3 iconic and widely loved French dishes:

* **Croissants:** Flaky, buttery pastries that are a breakfast staple in France.
* **Macarons:** Delicate, colorful cookies with a smooth, creamy filling.
* **Steak Frites:** A classic combination of grilled steak served with crispy fries. 


<end_of_turn>



**<font color='magenta'>Total time:</font>** 2.77 sec.

### Best attractions in Italy

In [24]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="best",
    number="five",
    items="attractions",
    place="Italy"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))  


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the best five attractions in Italy?


**<font color='green'>Answer:</font>**
It's impossible to definitively say which are the "best" five attractions in Italy, as personal preferences vary greatly. However, here are five incredibly popular and highly-rated attractions that are often considered must-sees:

* **Colosseum:** A massive ancient Roman amphitheater, a symbol of the Roman Empire and a testament to its engineering prowess.
* **Vatican City:** The smallest country in the world, home to St. Peter's Basilica, the Vatican Museums, and the Sistine Chapel.
* **Leaning Tower of Pisa:** A famous bell tower that leans at an impressive angle, a symbol of the city of Pisa.
* **Pompeii:** A well-preserved Roman city frozen in time by the eruption of Mount Vesuvius.
* **Cinque Terre:** A string of five colorful villages perched on cliffs overlooking the Ligurian Sea. 


<end_of_turn>



**<font color='magenta'>Total time:</font>** 5.1 sec.

### Most affordable places to retire in Portugal

In [25]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="most affordable",
    number="two",
    items="places to retire",
    place="Portugal"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the most affordable two places to retire in Portugal?


**<font color='green'>Answer:</font>**
* **Évora:** This historic city boasts a low cost of living, charming architecture, and a vibrant cultural scene.
* **Coimbra:** Home to the University of Coimbra, this city offers a rich academic atmosphere and a relatively affordable lifestyle. 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 1.66 sec.

### Less known but great places to stay in Romania

In [26]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="Less known but great",
    number="4",
    items="places to stay",
    place="Romania"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the Less known but great 4 places to stay in Romania?


**<font color='green'>Answer:</font>**
Here are some lesser-known but fantastic places to stay in Romania:

* **Sighisoara, Transylvania:**  A medieval citadel with cobblestone streets, charming guesthouses, and a rich history.
* **Bran Castle, Transylvania:**  The iconic castle made famous by Bram Stoker's Dracula. While it's a popular tourist destination, there are still charming guesthouses and hotels nearby.
* **The Danube Delta:**  A UNESCO World Heritage site, this unique ecosystem offers eco-friendly lodges and charming guesthouses.
* **Bucharest's Old Town:**  A vibrant and historic district with a mix of traditional and modern architecture, offering a range of boutique hotels and guesthouses.


Let me know if you'd like to explore other destinations in Romania! 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 4.61 sec.

### Great nature sights in USA

In [27]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="Great nature spots",
    number="4",
    items="to visit",
    place="USA"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the Great nature spots 4 to visit in USA?


**<font color='green'>Answer:</font>**
Here are some of the most incredible natural spots to visit in the USA:

* **Grand Canyon National Park, Arizona:**  A breathtaking canyon carved by the Colorado River, offering stunning views and hiking trails.
* **Yellowstone National Park, Wyoming, Montana, and Idaho:** Home to geysers, hot springs, and abundant wildlife, including bison, elk, and wolves.
* **Yosemite National Park, California:**  Famous for its towering granite cliffs, giant sequoia trees, and cascading waterfalls.
* **Acadia National Park, Maine:**  A coastal park with rugged mountains, rocky shores, and beautiful beaches. 


Let me know if you'd like to explore more natural wonders! 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 4.26 sec.

### Most important battles in WW2

In [28]:
t = time()
response = gemma_lm.generate(prompt.format(
    adjective="most important",
    number="5",
    items="battles",
    place="WW2"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.


**<font color='red'>Question:</font>** What are the most important 5 battles in WW2?


**<font color='green'>Answer:</font>**
Here are five of the most important battles of World War II, with a brief explanation of their significance:

* **Battle of Midway (June 1942):** A decisive naval victory for the United States against Japan, halting their advance in the Pacific. It marked a turning point in the war in the Pacific.
* **Battle of Stalingrad (August 1942 - February 1943):** A brutal and costly battle for the Soviet Union, but ultimately a turning point in the war against Germany. The German defeat at Stalingrad marked the beginning of their retreat from the Eastern Front.
* **Battle of El Alamein (October 1942):** A major victory for the British and Commonwealth forces against the Axis powers, halting the German advance in North Africa. It was a crucial turning point in the war in the Mediterranean.
* **Battle of Normandy (June 1944):** The largest amphibious invasion in history, the D-Day landings



**<font color='magenta'>Total time:</font>** 5.83 sec.

## Do it yourself questions

In [29]:
prompt = """
You are an AI assistant designed to answer questions about how to do 
various activities by yourself (do it yourself).
Return the answer formated nicely, for example step by step.
Question: How I can {action}?
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(
    action="build a shelter in the woods, if I have only a saw and a hammer?",
    ), 
    max_length=512)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions about how to do 
various activities by yourself (do it yourself).
Return the answer formated nicely, for example step by step.


**<font color='red'>Question:</font>** How I can build a shelter in the woods, if I have only a saw and a hammer??


**<font color='green'>Answer:</font>**
It's great you're thinking about shelter building!  However, building a safe and effective shelter with only a saw and a hammer is challenging.  Here's why:

**Why it's difficult:**

* **Saw limitations:** A saw is primarily for cutting wood, not shaping or building complex structures.
* **Hammer limitations:** A hammer is for driving nails, not creating structural supports.
* **Safety concerns:**  Without specialized tools and knowledge, building a shelter could be dangerous.

**What you can do:**

1. **Prioritize safety:**  Your top priority is safety.  Avoid building a shelter in unstable terrain or near potential hazards.
2. **Find natural materials:** Look for fallen branches, logs, and rocks that can be used for shelter building.
3. **Create a basic lean-to:**  Use fallen branches to create a lean-to against a tree or rock formation. This provides basic protection from the elements.
4. **Use natural materials for warmth:**  Gather dry leaves, moss, or pine needles to create a layer of insulation.
5. **Stay hydrated and conserve energy:**  This is crucial for survival.

**Important Considerations:**

* **Weather:**  Shelter building is more complex in harsh weather conditions.
* **Experience:**  Building a safe and effective shelter requires experience and knowledge.
* **Resources:**  You'll need to be resourceful and adapt to your surroundings.

**Remember:**  Building a shelter in the woods is a complex task.  It's best to prioritize safety and seek professional guidance if possible.


**Additional Resources:**

* **Outdoor survival guides:**  These can provide valuable information on shelter building techniques.
* **Local survival courses:**  Consider taking a course to learn from experienced instructors.
* **Online resources:**  There are many websites and forums dedicated to wilderness survival. 


Let me know if you have any other questions! 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 11.78 sec.

In [30]:
t = time()
response = gemma_lm.generate(prompt.format(
    action="scrap a dinner for my two kids with only 4 eggs, a pint of flour and some oil?",
    ), 
    max_length=512)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions about how to do 
various activities by yourself (do it yourself).
Return the answer formated nicely, for example step by step.


**<font color='red'>Question:</font>** How I can scrap a dinner for my two kids with only 4 eggs, a pint of flour and some oil??


**<font color='green'>Answer:</font>**
You can absolutely make a delicious dinner with those ingredients! Here's a simple recipe for **Egg Drop Soup** that's perfect for kids:

**Ingredients:**

* 4 eggs
* 1/2 cup flour
* 4 cups chicken broth (or vegetable broth)
* 1 tablespoon oil
* Salt and pepper to taste

**Instructions:**

1. **Prepare the broth:** In a large pot, bring the chicken broth to a boil.
2. **Make the egg mixture:** In a separate bowl, whisk the eggs until light and frothy. Gradually whisk in the flour until smooth.
3. **Add the egg mixture:** Slowly pour the egg mixture into the boiling broth, stirring constantly.
4. **Cook and serve:** Continue to cook for 2-3 minutes, or until the soup thickens. Season with salt and pepper to taste.
5. **Serve:** Ladle the soup into bowls and garnish with chopped green onions or sesame seeds (optional).

**Tips:**

* You can add other vegetables like chopped carrots or mushrooms to the soup for extra flavor and nutrition.
* For a richer flavor, use chicken broth instead of vegetable broth.
* You can also add a pinch of red pepper flakes for a little heat.

Enjoy this simple and satisfying meal!


Let me know if you'd like to try another recipe! 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 8.47 sec.

## Multiple steps reasoning

In [31]:
prompt = """
You are an AI assistant designed to answer questions in a chain of thought.
Use the answer to the first question as input for the second question.
Question one: What are the best city from {country} for tourists?
Question two: What are the best {number} of attractions from the city identified as answer to question one?
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(
    number="3",
    country="France"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions in a chain of thought.
Use the answer to the first question as input for the second question.
Question one: What are the best city from France for tourists?
Question two: What are the best 3 of attractions from the city identified as answer to question one?


**<font color='green'>Answer:</font>**
Question one: What are the best city from France for tourists?


**<font color='green'>Answer:</font>** Paris, Lyon, Nice, Marseille are some of the best cities for tourists in France. 

Question two: What are the best 3 of attractions from the city identified as answer to question one?


**<font color='green'>Answer:</font>** Paris is known for its iconic Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 2.42 sec.

In [32]:
prompt = """
You are an AI assistant designed to answer questions in a chain of thought.
Use the answer to the first question as input for the second question.
Question one: What are the best food item from {country}?
Question two: What are the most important {number} ingredients for the food identified at first step?
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(
    number="3",
    country="France"
    ), 
    max_length=256)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are an AI assistant designed to answer questions in a chain of thought.
Use the answer to the first question as input for the second question.
Question one: What are the best food item from France?
Question two: What are the most important 3 ingredients for the food identified at first step?


**<font color='green'>Answer:</font>**
Question one: What are the best food items from France?


**<font color='green'>Answer:</font>** This is subjective, but some of the most popular and highly regarded food items from France include:
* **Cheese:** France is renowned for its diverse range of cheeses, from the classic Brie and Camembert to the more pungent Roquefort and Comté.
* **Wine:** France is the world's leading producer of wine, with a wide variety of styles, from the crisp whites of Burgundy to the full-bodied reds of Bordeaux.
* **Bread:** French bread is known for its crusty exterior and soft interior, often made with a combination of flour, water, yeast, and salt.
* **Pastries:** From croissants and pain au chocolat to macarons and éclairs, French pastries are famous for their delicate flavors and beautiful presentation.
* **Seafood:** France has a long coastline and a rich tradition of seafood, with



**<font color='magenta'>Total time:</font>** 5.23 sec.

# Some really difficult reasoning

In [33]:
prompt = """
You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h. 
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth 
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti. 
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.
Reasoning: Think step by step. Explain your reasoning.
Question: {question}
Answer:
"""
t = time()
response = gemma_lm.generate(prompt.format(question="How many kilometers will travel totally the swallow?"), 
    max_length=512)
display(Markdown(colorize_text(f"{response}\n\nTotal time: {round(time()-t, 2)} sec.")))


You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h. 
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth 
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti. 
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.


**<font color='blue'>Reasoning:</font>** Think step by step. Explain your reasoning.


**<font color='red'>Question:</font>** How many kilometers will travel totally the swallow?


**<font color='green'>Answer:</font>**
Let's break down the problem step-by-step:

1. **Train's journey:** The train travels 60 km from Bucharest to Ploiesti.

2. **Swallow's first flight:** The swallow flies from Ploiesti to the train at 90 km/h.  This flight is 90 km long.

3. **Swallow's second flight:** The swallow flies back from the train to Ploiesti at 90 km/h. This flight is 90 km long.

4. **Total distance:** The swallow flies a total of 90 km + 90 km = 180 km.


**Therefore, the swallow will travel a total of 180 kilometers.**


**

**<font color='darkblue'>Explanation:</font>****

The swallow's flight is not affected by the train's speed. The swallow flies at a constant speed relative to the ground. 
<end_of_turn>



**<font color='magenta'>Total time:</font>** 5.74 sec.

# Conclusions


We were using `gemma2_instruct_2b_en`.

Here is the analysis of results with gemma2_instruct_2b_en:

* General question: answer to the point, and the answer seems sensible. 
* General history knowledge is definitely improved over Gemma. American history (very simple questions) were aced.
* Math problem, with answer as Python code: correct, and code is documented.
* More Python code: if too complex, start to make mistakes.
* Software design questions: was able to define correctly a phone agenda class and an REST API service (using Flask)
* C++ and code of decent quality.
* Multi-parameters question: all answers are correct.
* A chain of reasoning (one task receives as input the output of the previous task) was performed correctly.
* More difficult math problem, requiring reasoning: the answer is not correct, because the reasoning is not correct. Gemma or Llama3 were not better at it, either.